In [1]:
import polars as pl
import csv
import glob
import os

In [82]:
def clean_file(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
        reader = csv.reader(infile, delimiter=";", quotechar='"', escapechar="\\")
        writer = csv.writer(outfile, delimiter=";", quotechar='"', escapechar="\\", quoting=csv.QUOTE_MINIMAL)
        
        for row in reader:
            if row.count('"') % 2 != 0:  
                continue
            max_columns = 24  
            row = row[:max_columns] + [""] * (max_columns - len(row))
            writer.writerow(row)

In [2]:
extract_folder = "extract"
cleaned_folder = "cleaned"

In [3]:
dat_files = glob.glob(f"{extract_folder}/*.dat")
print(f"Number of .dat files found: {len(dat_files)}")

Number of .dat files found: 25176


In [85]:
os.makedirs(cleaned_folder, exist_ok=True)

dat_files = glob.glob(f"{extract_folder}/*.dat")

for file in dat_files:
    cleaned_path = os.path.join(cleaned_folder, os.path.basename(file))
    clean_file(file, cleaned_path)

In [4]:
df_list = []
df_schema = {
    'recordType': pl.Utf8,
    'districtCode': pl.Utf8,
    'propertyId': pl.Utf8,
    'saleCounter': pl.Utf8,
    'downloadDatetime': pl.Utf8,
    'propertyName': pl.Utf8,
    'propertyUnitNumber': pl.Utf8,
    'propertyHouseNumber': pl.Utf8,
    'propertyStreetName': pl.Utf8,
    'propertyLocality': pl.Utf8,
    'propertyPostCode': pl.Utf8,
    'area': pl.Utf8,
    'areaType': pl.Utf8,
    'contractDate': pl.Utf8,
    'settlementDate': pl.Utf8,
    'purchasePrice': pl.Utf8,
    'zoning': pl.Utf8,
    'natureofProperty': pl.Utf8,
    'primaryPurpose': pl.Utf8,
    'strataLotNumber': pl.Utf8,
    'componentCode': pl.Utf8,
    'saleCode': pl.Utf8,
    'InterestofSale%': pl.Utf8,
    'dealingNumber': pl.Utf8
}

In [5]:
dat_cleaned = glob.glob(f"{cleaned_folder}/*.dat")
for file in dat_cleaned:
    try:
        df = pl.read_csv(
            file,
            separator=";",
            has_header=False,
            schema = df_schema,
            truncate_ragged_lines=True,
            ignore_errors=True
        ).with_columns(
            [pl.col(column).cast(pl.Utf8) for column in df_schema.keys()])
        df_list.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

In [6]:
if df_list:
    try:
        df_combined = pl.concat(df_list)
        df_filtered = df_combined.filter(pl.col("recordType") == "B")
        df_filtered.write_csv("filtered_data.csv")
        print("Filtered data saved to 'filtered_data.csv'.")
    except Exception as e:
        print(f"Error during concatenation or filtering: {e}")
else:
    print("No valid DataFrames to process.")

Filtered data saved to 'filtered_data.csv'.


In [7]:
df_filtered.head()

recordType,districtCode,propertyId,saleCounter,downloadDatetime,propertyName,propertyUnitNumber,propertyHouseNumber,propertyStreetName,propertyLocality,propertyPostCode,area,areaType,contractDate,settlementDate,purchasePrice,zoning,natureofProperty,primaryPurpose,strataLotNumber,componentCode,saleCode,InterestofSale%,dealingNumber
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""B""","""001""","""2857799""","""1""","""20240101 01:07""",null,null,"""176""","""LAKE RD""","""ELRINGTON""","""2325""","""25.15""","""H""","""20231219""","""20231222""","""1330000""","""RU2""","""R""","""RESIDENCE""",null,"""RAN""",null,"""0""","""AT729586 """
"""B""","""001""","""4228""","""2""","""20240101 01:07""",null,null,"""2""","""KING ST""","""BRANXTON""","""2335""","""1864""","""M""","""20231115""","""20231222""","""850000""","""R3""","""R""","""RESIDENCE""",null,"""MAB""",null,"""0""","""AT731473 """
"""B""","""001""","""7750""","""3""","""20240101 01:07""",null,null,"""275""","""MAITLAND RD""","""CESSNOCK""","""2325""","""379.8""","""M""","""20231026""","""20231222""","""500000""","""R3""","""3""","""SHOP""",null,"""MAA""",null,"""0""","""AT729914 """
"""B""","""001""","""11439""","""4""","""20240101 01:07""",null,null,"""19""","""DEAKIN ST""","""KURRI KURRI""","""2327""","""1682""","""M""","""20231103""","""20231222""","""575000""","""R2""","""R""","""RESIDENCE""",null,"""AAO""",null,"""0""","""AT731374 """
"""B""","""001""","""18319""","""5""","""20240101 01:07""",null,null,"""700""","""MILSONS ARM RD""","""WOLLOMBI""","""2325""","""31.77""","""H""","""20231218""","""20231222""","""236667""","""RU2""","""R""","""RESIDENCE""",null,"""RAG""",null,"""0""","""AT733421 """


In [25]:
df_cleaning = df_filtered.with_columns(
    pl.col("contractDate").str.strptime(pl.Date, "%Y%m%d", strict=False).alias("contractDate")
)

In [26]:
df_cleaning = df_cleaning.select('districtCode',
                            'propertyId',
                            'propertyName',
                            'propertyUnitNumber',
                            'propertyStreetName',
                            'propertyLocality',
                            'propertyPostCode',
                            'area',
                            'areaType',
                            'contractDate',
                            'settlementDate',
                            'purchasePrice',
                            'primaryPurpose',
                            'strataLotNumber')

In [27]:
df_cleaning = df_cleaning.with_columns(pl.col("area").cast(pl.Float64).alias("area"))

In [28]:
df_cleaning = df_cleaning.with_columns(
    pl.when(pl.col('areaType') == 'M')
     .then(pl.col('area').cast(pl.Float64))
     .when(pl.col('areaType') == 'H')
     .then(pl.col('area').cast(pl.Float64) * 10000)
     .otherwise(None)
     .alias('areaM')
)

In [29]:
df_cleaning = df_cleaning.with_columns(pl.col("purchasePrice").cast(pl.Int64).alias("purchasePrice"))

In [30]:
df_cleaning = df_cleaning.with_columns(pl.col("propertyId").cast(pl.Int64).alias("propertyId"))

In [31]:
df_cleaning = df_cleaning.with_columns(
    pl.col("settlementDate").str.strptime(pl.Date, "%Y%m%d", strict=False).alias("settlementDate")
)

In [32]:
df_cleaning = df_cleaning.with_columns([
    pl.col('settlementDate').dt.year().alias('settlementYear'),
    pl.col('settlementDate').dt.month().alias('settlementMonth')
])


In [33]:
df_cleaning = df_cleaning.filter(
    pl.col('settlementYear') >= 2021
)

In [34]:
df_cleaning = df_cleaning.with_columns((pl.col("purchasePrice") / pl.col("areaM")).alias("pricePerM2").round(2))

In [35]:
df_cleaning.head()

districtCode,propertyId,propertyName,propertyUnitNumber,propertyStreetName,propertyLocality,propertyPostCode,area,areaType,contractDate,settlementDate,purchasePrice,primaryPurpose,strataLotNumber,areaM,settlementYear,settlementMonth,pricePerM2
str,i64,str,str,str,str,str,f64,str,date,date,i64,str,str,f64,i32,i8,f64
"""001""",2857799,null,null,"""LAKE RD""","""ELRINGTON""","""2325""",25.15,"""H""",2023-12-19,2023-12-22,1330000,"""RESIDENCE""",null,251500.0,2023,12,5.29
"""001""",4228,null,null,"""KING ST""","""BRANXTON""","""2335""",1864.0,"""M""",2023-11-15,2023-12-22,850000,"""RESIDENCE""",null,1864.0,2023,12,456.01
"""001""",7750,null,null,"""MAITLAND RD""","""CESSNOCK""","""2325""",379.8,"""M""",2023-10-26,2023-12-22,500000,"""SHOP""",null,379.8,2023,12,1316.48
"""001""",11439,null,null,"""DEAKIN ST""","""KURRI KURRI""","""2327""",1682.0,"""M""",2023-11-03,2023-12-22,575000,"""RESIDENCE""",null,1682.0,2023,12,341.85
"""001""",18319,null,null,"""MILSONS ARM RD""","""WOLLOMBI""","""2325""",31.77,"""H""",2023-12-18,2023-12-22,236667,"""RESIDENCE""",null,317700.0,2023,12,0.74


In [36]:
df_cleaning_unique = df_cleaning.unique(keep="last")

In [37]:
df_cleaning_unique.n_unique()

792941

In [38]:
cleaned_df = df_cleaning_unique.with_columns(pl.arange(0, df_cleaning_unique.height).alias("uniqueSaleKey"))

In [39]:
cleaned_df.head()

districtCode,propertyId,propertyName,propertyUnitNumber,propertyStreetName,propertyLocality,propertyPostCode,area,areaType,contractDate,settlementDate,purchasePrice,primaryPurpose,strataLotNumber,areaM,settlementYear,settlementMonth,pricePerM2,uniqueSaleKey
str,i64,str,str,str,str,str,f64,str,date,date,i64,str,str,f64,i32,i8,f64,i64
"""005""",4100064,null,null,"""SKYLARK AVE""","""THORNTON""","""2322""",null,null,2021-11-05,2021-12-17,571000,"""RESIDENCE""","""11""",null,2021,12,null,0
"""243""",2645282,null,null,"""LEONARD ST""","""HAY""","""2711""",1454.0,"""M""",2023-10-23,2023-11-20,315000,"""RESIDENCE""",null,1454.0,2023,11,216.64,1
"""214""",2193486,null,null,"""KASTELAN ST""","""BLACKTOWN""","""2148""",581.7,"""M""",2022-11-04,2023-01-20,810000,"""RESIDENCE""",null,581.7,2023,1,1392.47,2
"""272""",4514768,null,null,"""JUMPING CREEK RD""","""GREENLEIGH""","""2620""",870.0,"""M""",2021-10-10,2023-11-27,479000,"""VACANT LAND""",null,870.0,2023,11,550.57,3
"""529""",4517402,null,null,"""SPEEDWAY RD""","""BOXERS CREEK""","""2580""",2.901,"""H""",2023-08-03,2023-11-01,44590,"""VACANT LAND""",null,29010.0,2023,11,1.54,4


In [40]:
output_file = "./cleaned.csv"

cleaned_df.write_csv(output_file)

In [41]:
unit_df = cleaned_df.filter((pl.col("propertyUnitNumber") != "null") | (pl.col("strataLotNumber") != "null"))

In [42]:
unit_df.head()

districtCode,propertyId,propertyName,propertyUnitNumber,propertyStreetName,propertyLocality,propertyPostCode,area,areaType,contractDate,settlementDate,purchasePrice,primaryPurpose,strataLotNumber,areaM,settlementYear,settlementMonth,pricePerM2,uniqueSaleKey
str,i64,str,str,str,str,str,f64,str,date,date,i64,str,str,f64,i32,i8,f64,i64
"""005""",4100064,null,null,"""SKYLARK AVE""","""THORNTON""","""2322""",null,null,2021-11-05,2021-12-17,571000,"""RESIDENCE""","""11""",null,2021,12,null,0
"""088""",4175060,null,"""109""","""WEST ST""","""CROWS NEST""","""2065""",79.0,"""M""",2020-12-07,2021-01-18,700000,"""RESIDENCE""","""27""",79.0,2021,1,8860.76,6
"""708""",4535501,null,"""78 B""","""BARANGAROO AVE""","""BARANGAROO""","""2000""",332.0,"""M""",2022-12-23,2024-05-23,18272534,"""RESIDENCE""","""291""",332.0,2024,5,55037.75,7
"""708""",2117528,"""PARKSIDE""","""7 A""","""COOK RD""","""CENTENNIAL PARK""","""2021""",null,null,2024-04-08,2024-04-24,1100000,"""RESIDENCE""","""13""",null,2024,4,null,8
"""137""",3902516,null,"""1205""","""VICTORIA ST""","""BURWOOD""","""2134""",83.0,"""M""",2021-04-02,2021-05-14,709000,"""RESIDENCE""","""61""",83.0,2021,5,8542.17,10


In [43]:
unit_df_filter = unit_df.filter(pl.col("primaryPurpose").is_in(["UNIT", "RESIDENCE", "HOUSE UNIT"]))

In [44]:
unit_df_filter.head()

districtCode,propertyId,propertyName,propertyUnitNumber,propertyStreetName,propertyLocality,propertyPostCode,area,areaType,contractDate,settlementDate,purchasePrice,primaryPurpose,strataLotNumber,areaM,settlementYear,settlementMonth,pricePerM2,uniqueSaleKey
str,i64,str,str,str,str,str,f64,str,date,date,i64,str,str,f64,i32,i8,f64,i64
"""005""",4100064,null,null,"""SKYLARK AVE""","""THORNTON""","""2322""",null,null,2021-11-05,2021-12-17,571000,"""RESIDENCE""","""11""",null,2021,12,null,0
"""088""",4175060,null,"""109""","""WEST ST""","""CROWS NEST""","""2065""",79.0,"""M""",2020-12-07,2021-01-18,700000,"""RESIDENCE""","""27""",79.0,2021,1,8860.76,6
"""708""",4535501,null,"""78 B""","""BARANGAROO AVE""","""BARANGAROO""","""2000""",332.0,"""M""",2022-12-23,2024-05-23,18272534,"""RESIDENCE""","""291""",332.0,2024,5,55037.75,7
"""708""",2117528,"""PARKSIDE""","""7 A""","""COOK RD""","""CENTENNIAL PARK""","""2021""",null,null,2024-04-08,2024-04-24,1100000,"""RESIDENCE""","""13""",null,2024,4,null,8
"""137""",3902516,null,"""1205""","""VICTORIA ST""","""BURWOOD""","""2134""",83.0,"""M""",2021-04-02,2021-05-14,709000,"""RESIDENCE""","""61""",83.0,2021,5,8542.17,10


In [45]:
unit_df_filter.n_unique()

258602

In [46]:
output_file2 = "./units.csv"

unit_df_filter.write_csv(output_file2)

In [47]:
unit_groupby = unit_df_filter.group_by(['propertyLocality', 'settlementYear']).agg(pl.col('pricePerM2').mean().alias('avgPricePerM2').round(2))
unit_groupby.sort(['propertyLocality', 'settlementYear'])

propertyLocality,settlementYear,avgPricePerM2
str,i32,f64
"""ABBOTSBURY""",2021,2653.68
"""ABBOTSFORD""",2021,6215.51
"""ABBOTSFORD""",2022,5805.39
"""ABBOTSFORD""",2023,10195.46
"""ABBOTSFORD""",2024,9810.86
…,…,…
"""YOWIE BAY""",2024,5393.94
"""ZETLAND""",2021,10411.85
"""ZETLAND""",2022,10388.3


In [48]:
output_file3 = "./units_grouped.csv"

unit_groupby.write_csv(output_file3)

In [49]:
df = pl.read_csv("units_grouped.csv")

In [50]:
unit_pivot_df = df.pivot(
    index="propertyLocality",
    columns="settlementYear",
    values="avgPricePerM2",
    aggregate_function=None  # Use this if you need to aggregate duplicate entries
)

C:\Users\mtklu\AppData\Local\Temp\ipykernel_7068\2272853186.py:1: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  unit_pivot_df = df.pivot(


In [51]:
unit_pivot_df = unit_pivot_df[["propertyLocality", "2021", "2022", "2023", "2024"]]

In [52]:
unit_pivot_df.head()

propertyLocality,2021,2022,2023,2024
str,f64,f64,f64,f64
"""URALLA""",787.68,1224.2,808.32,966.29
"""PORT KEMBLA""",3074.48,3499.25,4097.44,3644.48
"""SURF BEACH""",4056.58,2545.69,2000.53,null
"""MANGERTON""",2448.88,null,2676.06,2423.31
"""BATEMANS BAY""",3306.23,5373.02,5790.62,5071.56


In [53]:
output_file_units = "Data/units_data.csv"

unit_pivot_df.write_csv(output_file_units)

In [54]:
house_df = cleaned_df.filter(
    (pl.col("propertyUnitNumber").is_null() & pl.col("strataLotNumber").is_null()))

In [55]:
house_df.head()

districtCode,propertyId,propertyName,propertyUnitNumber,propertyStreetName,propertyLocality,propertyPostCode,area,areaType,contractDate,settlementDate,purchasePrice,primaryPurpose,strataLotNumber,areaM,settlementYear,settlementMonth,pricePerM2,uniqueSaleKey
str,i64,str,str,str,str,str,f64,str,date,date,i64,str,str,f64,i32,i8,f64,i64
"""243""",2645282,null,null,"""LEONARD ST""","""HAY""","""2711""",1454.0,"""M""",2023-10-23,2023-11-20,315000,"""RESIDENCE""",null,1454.0,2023,11,216.64,1
"""214""",2193486,null,null,"""KASTELAN ST""","""BLACKTOWN""","""2148""",581.7,"""M""",2022-11-04,2023-01-20,810000,"""RESIDENCE""",null,581.7,2023,1,1392.47,2
"""272""",4514768,null,null,"""JUMPING CREEK RD""","""GREENLEIGH""","""2620""",870.0,"""M""",2021-10-10,2023-11-27,479000,"""VACANT LAND""",null,870.0,2023,11,550.57,3
"""529""",4517402,null,null,"""SPEEDWAY RD""","""BOXERS CREEK""","""2580""",2.901,"""H""",2023-08-03,2023-11-01,44590,"""VACANT LAND""",null,29010.0,2023,11,1.54,4
"""575""",4389437,null,null,"""KINCAID ST""","""WAGGA WAGGA""","""2650""",132.3,"""M""",2024-01-11,2024-01-25,685000,"""RESIDENCE""",null,132.3,2024,1,5177.63,5


In [56]:
house_df_filter = house_df.filter(pl.col("primaryPurpose").is_in(["RESIDENCE", "DWELLING", "HOUSE AND FARM"]))

In [57]:
house_df_filter.head()

districtCode,propertyId,propertyName,propertyUnitNumber,propertyStreetName,propertyLocality,propertyPostCode,area,areaType,contractDate,settlementDate,purchasePrice,primaryPurpose,strataLotNumber,areaM,settlementYear,settlementMonth,pricePerM2,uniqueSaleKey
str,i64,str,str,str,str,str,f64,str,date,date,i64,str,str,f64,i32,i8,f64,i64
"""243""",2645282,null,null,"""LEONARD ST""","""HAY""","""2711""",1454.0,"""M""",2023-10-23,2023-11-20,315000,"""RESIDENCE""",null,1454.0,2023,11,216.64,1
"""214""",2193486,null,null,"""KASTELAN ST""","""BLACKTOWN""","""2148""",581.7,"""M""",2022-11-04,2023-01-20,810000,"""RESIDENCE""",null,581.7,2023,1,1392.47,2
"""575""",4389437,null,null,"""KINCAID ST""","""WAGGA WAGGA""","""2650""",132.3,"""M""",2024-01-11,2024-01-25,685000,"""RESIDENCE""",null,132.3,2024,1,5177.63,5
"""082""",631178,null,null,"""BROOKLYN RD""","""BROOKLYN""","""2083""",223.1,"""M""",2024-09-16,2024-10-28,725000,"""RESIDENCE""",null,223.1,2024,10,3249.66,9
"""219""",4238849,null,null,"""CASTLEREAGH RD""","""WILBERFORCE""","""2756""",903.6,"""M""",2023-12-24,2024-03-25,1465000,"""RESIDENCE""",null,903.6,2024,3,1621.29,12


In [58]:
house_df_filter.n_unique()

419495

In [59]:
output_file4 = "./house.csv"

house_df_filter.write_csv(output_file4)

In [60]:
house_groupby = house_df_filter.group_by(['propertyLocality', 'settlementYear']).agg(pl.col('pricePerM2').mean().alias('avgPricePerM2').round(2))
house_groupby.sort(['propertyLocality', 'settlementYear'])


propertyLocality,settlementYear,avgPricePerM2
str,i32,f64
"""AARONS PASS""",2022,1.35
"""AARONS PASS""",2024,1.0
"""ABBOTSBURY""",2021,1781.23
"""ABBOTSBURY""",2022,2318.06
"""ABBOTSBURY""",2023,2203.31
…,…,…
"""ZARA""",2024,7.09
"""ZETLAND""",2021,13354.66
"""ZETLAND""",2022,11080.24


In [61]:
house_groupby.head()

propertyLocality,settlementYear,avgPricePerM2
str,i32,f64
"""JEWELLS""",2024,1788.29
"""THE PINNACLES""",2024,1.92
"""BOORABEE PARK""",2022,9.59
"""BURRIER""",2021,3.46
"""ELLENBOROUGH""",2023,16.11


In [62]:
output_file5 = "./house_grouped.csv"

house_groupby.write_csv(output_file5)

In [63]:
df = pl.read_csv("house_grouped.csv")

In [64]:
house_pivot_df = df.pivot(
    index="propertyLocality",
    columns="settlementYear",
    values="avgPricePerM2",
    aggregate_function=None  # Use this if you need to aggregate duplicate entries
)

C:\Users\mtklu\AppData\Local\Temp\ipykernel_7068\230322041.py:1: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  house_pivot_df = df.pivot(


In [65]:
house_pivot_df = house_pivot_df[["propertyLocality", "2021", "2022", "2023", "2024"]]

In [66]:
house_pivot_df.head()

propertyLocality,2021,2022,2023,2024
str,f64,f64,f64,f64
"""JEWELLS""",1063.33,1444.19,1767.65,1788.29
"""THE PINNACLES""",5.19,3.92,null,1.92
"""BOORABEE PARK""",5.4,9.59,14.17,10.66
"""BURRIER""",3.46,null,null,null
"""ELLENBOROUGH""",71.87,0.47,16.11,15.14


In [67]:
output_file_units = "Data/houses_data.csv"

house_pivot_df.write_csv(output_file_units)